In [1]:
import anndata as ad
import pandas as pd
import numpy as np

In [2]:
############################################
# Config / Inputs
############################################
# The file that currently has Symbols and CellTypist labels, but 0% MT
SRC_H5AD = "../../celltypist/fujita_celltypist_GPU_counts_only.h5ad"
############################################
# 1. Open Source (Backed)
############################################
print(f"📖 Opening {SRC_H5AD} in backed mode...")
adata = ad.read_h5ad(SRC_H5AD, backed="r")

📖 Opening ../../celltypist/fujita_celltypist_GPU_counts_only.h5ad in backed mode...


In [3]:
adata.obs.columns

Index(['sample', 'Unnamed: 0', 'grouping.by', 'cell.type', 'individualID',
       'seurat.object', 'projid', 'Study', 'msex', 'educ', 'race', 'spanish',
       'apoe_genotype', 'age_at_visit_max', 'age_first_ad_dx', 'age_death',
       'cts_mmse30_first_ad_dx', 'cts_mmse30_lv', 'pmi', 'braaksc', 'ceradsc',
       'cogdx', 'dcfdx_lv', 'n_genes_by_counts', 'log1p_n_genes_by_counts',
       'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes',
       'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt',
       'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo',
       'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier',
       'mt_outlier', 'n_genes', 'doublet_scores', 'predicted_doublets',
       'predicted_labels', 'over_clustering', 'majority_voting', 'conf_score',
       'celltypist_cell_label', 'celltypist_conf_score'],
      dtype='object')

In [4]:
# Convert just the QC columns to a real dataframe to see the values
qc_check = adata.obs[['total_counts', 'pct_counts_mt', 'celltypist_cell_label']].head(10)
print(qc_check)

# Check if they are all NaN
print("\nIs pct_counts_mt empty?")
print(adata.obs['pct_counts_mt'].isna().value_counts())

                                total_counts  pct_counts_mt  \
190403-B4-A_ACTACGAAGCACCCAC-1       68036.0            0.0   
190403-B4-A_ACCAAACGTTATTCTC-1       62788.0            0.0   
190403-B4-A_CCTCACATCACCGGTG-1       52072.0            0.0   
190403-B4-A_GCATCTCGTCAACCTA-1       51142.0            0.0   
190403-B4-A_TATCGCCTCTGTCGCT-1       48090.0            0.0   
190403-B4-A_CACAACAAGCTACTGT-1       47790.0            0.0   
190403-B4-A_CCCAACTAGCTCGACC-1       45960.0            0.0   
190403-B4-A_TGATTTCTCTCAGAAC-1       45914.0            0.0   
190403-B4-A_AGAACAATCCGTCCTA-1       45194.0            0.0   
190403-B4-A_CTCCACAAGCACACCC-1       43189.0            0.0   

                                  celltypist_cell_label  
190403-B4-A_ACTACGAAGCACCCAC-1     L6 OPRK1 THEMIS RGS6  
190403-B4-A_ACCAAACGTTATTCTC-1   L2-3 CUX2 NTNG1 COL5A2  
190403-B4-A_CCTCACATCACCGGTG-1   L2-3 CUX2 NTNG1 COL5A2  
190403-B4-A_GCATCTCGTCAACCTA-1   L2-3 CUX2 NTNG1 COL5A2  
190403-B4-A_TATC

In [5]:
# changed the names after running QC so mt QC showed 0 values

In [6]:
############################################
# 2. Fix Metadata in Memory (The QC Fix)
############################################
print("🧬 Recalculating MT/Ribo/HB metrics on Symbols...")

# Identify genes
mt_genes = adata.var_names[adata.var_names.str.startswith("MT-")].tolist()
ribo_genes = adata.var_names[adata.var_names.str.startswith(("RPS", "RPL"))].tolist()
hb_genes = adata.var_names[adata.var_names.str.contains("^HB[^(P)]")].tolist()

# Load ONLY these columns into RAM (safe for 1.3M cells)
# We calculate sums directly to keep RAM usage minimal
obs_fixed = adata.obs.copy()

print(f"   - Summing {len(mt_genes)} MT genes...")
obs_fixed["total_counts_mt"] = np.ravel(adata[:, mt_genes].to_memory().X.sum(axis=1))

print(f"   - Summing {len(ribo_genes)} Ribo genes...")
obs_fixed["total_counts_ribo"] = np.ravel(adata[:, ribo_genes].to_memory().X.sum(axis=1))

print(f"   - Summing {len(hb_genes)} HB genes...")
obs_fixed["total_counts_hb"] = np.ravel(adata[:, hb_genes].to_memory().X.sum(axis=1))

# Recalculate percentages
print("   - Updating percentages...")
obs_fixed["pct_counts_mt"] = 100 * obs_fixed["total_counts_mt"] / obs_fixed["total_counts"]
obs_fixed["pct_counts_ribo"] = 100 * obs_fixed["total_counts_ribo"] / obs_fixed["total_counts"]
obs_fixed["pct_counts_hb"] = 100 * obs_fixed["total_counts_hb"] / obs_fixed["total_counts"]

# Fill NaNs with 0 (for cells with 0 total counts, if any)
obs_fixed[["pct_counts_mt", "pct_counts_ribo", "pct_counts_hb"]] = \
    obs_fixed[["pct_counts_mt", "pct_counts_ribo", "pct_counts_hb"]].fillna(0)

############################################
# 3. Final Alignment Checks
############################################
print("\n🔎 Distribution Check (Post-Fix):")
print(obs_fixed[["pct_counts_mt", "pct_counts_ribo"]].describe().loc[['mean', 'max']])

🧬 Recalculating MT/Ribo/HB metrics on Symbols...
   - Summing 13 MT genes...
   - Summing 109 Ribo genes...
   - Summing 12 HB genes...
   - Updating percentages...

🔎 Distribution Check (Post-Fix):
      pct_counts_mt  pct_counts_ribo
mean       1.327024         1.053944
max       12.937063        25.000000


In [7]:
from scipy.stats import median_abs_deviation

# --- Calculate MAD Thresholds ---
def is_outlier(data, nmads):
    m = np.median(data)
    s = median_abs_deviation(data)
    return (data < m - nmads * s) | (data > m + nmads * s)

# Create the outlier masks
mt_mad_outlier = is_outlier(obs_fixed["pct_counts_mt"], 3)
mt_hard_outlier = obs_fixed["pct_counts_mt"] > 8.0

# Combine them: True if a cell is an outlier by EITHER rule
obs_fixed["mt_outlier"] = mt_mad_outlier | mt_hard_outlier

# --- Create the Keep Mask ---
# We keep cells that are NOT outliers
keep_mask = ~obs_fixed["mt_outlier"]

print(f"Total cells before filtering: {len(obs_fixed):,}")
print(f"Cells flagged as MT outliers: {obs_fixed['mt_outlier'].sum():,}")
print(f"Cells remaining: {keep_mask.sum():,}")

Total cells before filtering: 1,304,391
Cells flagged as MT outliers: 173,878
Cells remaining: 1,130,513


In [ ]:
# 1. Update the metadata for the cells we are keeping
obs_cleaned = obs_fixed[keep_mask].copy()

# 2. Slice the backed object and assemble the new shell
# Slicing a backed object creates a 'View' that streams only the kept cells
adata_filtered = ad.AnnData(
    X=adata[keep_mask, :].X,
    obs=obs_cleaned,
    var=adata.var,
    obsm=adata[keep_mask, :].obsm,
    layers=adata[keep_mask, :].layers,
    uns=adata.uns
)

# 3. Write to a new H5AD
output_path = "fujita_final_QC_filtered_symbols.h5ad"
print(f"🚀 Streaming filtered data to {output_path}...")
adata_filtered.write_h5ad(output_path, compression="gzip")

# 4. Close the original file handle
if hasattr(adata.file, "close"):
    adata.file.close()

print("✅ Done! File saved and filtered.")